Setting hyperparameters (hidden layer size, dropout probability, learning rate) using scorer_val dataset. Partly adapted from https://github.com/chiyuzhang94/hyperpartisan-ubc/blob/master/pytorch/bilstm_optim.py

In [0]:
! pip install sacremoses
import torch
import torch.nn.functional as F
import torch.autograd as autograd
import torch.nn as nn
from torch.utils.data import Dataset
#from utils.sensation_config import *
import numpy as np
import argparse
import logging
import pickle
import pandas as pd
import sacremoses
from torch.utils.data import dataloader, Dataset
from google.colab import drive
from tqdm import tqdm
drive.mount('/content/drive')
#from utils.global_variables import *

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
dataset = '/content/drive/My Drive/dataset/dataset'

# Set hyperparameters

In [0]:
# Hyper Parameters
batch_size = 50
sequence_length = 61
embedding_dim = 300
num_epoch = 5
num_layers = 1
num_classes = 2
bidirectional = True

In [0]:
# Parameters we want to fine-tune
op_parameters = []
var_dic = {}
var_dic['var_name'] = 'lr' # learning rate
var_dic['var_ini'] = 0.01
var_dic['var_range'] = [0.1,0.01,0.001,0.0001]
op_parameters.append(var_dic)

var_dic = {}
var_dic['var_name'] = "hidden_dim"
var_dic['var_ini'] = 100
var_dic['var_range'] = range(50,200,50) # range(100,501,100)
op_parameters.append(var_dic)

var_dic = {}
var_dic['var_name'] = "dropout"
var_dic['var_ini'] = 0.1
var_dic['var_range'] = [0.0,0.1,0.3,0.5,0.7,0.9]
op_parameters.append(var_dic)

In [0]:
# Load pickle files
scorer_val = pd.read_pickle(dataset+'/scorer_val.pickle')

In [0]:
data_size = scorer_val.shape[0]

In [0]:
# Split into train and test 
def split_dataset(dataset):
    train_size = int(dataset.shape[0]*0.8)
    train_df = dataset[:train_size]
    test_df = dataset[train_size:]
    return train_df, test_df
scorer_val_train, scorer_val_test = split_dataset(scorer_val)
train_texts, train_labels = list(scorer_val_train.title), list(scorer_val_train.hyperpartisan)
test_texts, test_labels = list(scorer_val_test.title), list(scorer_val_test.hyperpartisan)

In [0]:
train_size = len(train_texts)
test_size = len(test_texts)

# Load GloVe

In [0]:
# Load GloVe
!wget https://docs.google.com/uc?id=1KMJTagaVD9hFHXFTPtNk0u2JjvNlyCAu -O glove_split.aa
!wget https://docs.google.com/uc?id=1LF2yD2jToXriyD-lsYA5hj03f7J3ZKaY -O glove_split.ab
!wget https://docs.google.com/uc?id=1N1xnxkRyM5Gar7sv4d41alyTL92Iip3f -O glove_split.ac
!cat glove_split.?? > 'glove.6B.300d__50k.txt'
def load_glove(glove_path, embedding_dim):
    with open(glove_path) as f:
        token_ls = [PAD_TOKEN, UNK_TOKEN]
        embedding_ls = [np.zeros(embedding_dim), np.random.rand(embedding_dim)]
        for line in f:
            token, raw_embedding = line.split(maxsplit=1)
            token_ls.append(token)
            embedding = np.array([float(x) for x in raw_embedding.split()])
            embedding_ls.append(embedding)
        embeddings = np.array(embedding_ls)
    return token_ls, embeddings

PAD_TOKEN = '<PAD>'
UNK_TOKEN = '<UNK>'
glove_path = "glove.6B.300d__50k.txt"

vocab, embeddings = load_glove(glove_path, embedding_dim)

--2020-05-11 21:04:40--  https://docs.google.com/uc?id=1KMJTagaVD9hFHXFTPtNk0u2JjvNlyCAu
Resolving docs.google.com (docs.google.com)... 74.125.195.138, 74.125.195.100, 74.125.195.139, ...
Connecting to docs.google.com (docs.google.com)|74.125.195.138|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-0k-0g-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/313hbk9iut40n40f84qvjae35iihlc79/1589231025000/14514704803973256873/*/1KMJTagaVD9hFHXFTPtNk0u2JjvNlyCAu [following]
--2020-05-11 21:04:47--  https://doc-0k-0g-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/313hbk9iut40n40f84qvjae35iihlc79/1589231025000/14514704803973256873/*/1KMJTagaVD9hFHXFTPtNk0u2JjvNlyCAu
Resolving doc-0k-0g-docs.googleusercontent.com (doc-0k-0g-docs.googleusercontent.com)... 173.194.202.132, 2607:f8b0:400e:c00::84
Connecting to doc-0k-0g-docs.googleusercontent.com (doc-0k-0g-docs.googleusercontent.com)|173.194.20

# Featurize data

In [0]:
# Featurzie data
def featurize(data, labels, tokenizer, vocab, max_seq_length=128):
    vocab_to_idx = {word: i for i, word in enumerate(vocab)}
    text_data = []
    label_data = []
    for ex in tqdm(data):
        tokenized = tokenizer.tokenize(ex)
        ids = [vocab_to_idx.get(token, 1) for token in tokenized]
        text_data.append(ids)
    return text_data, labels

tokenizer = sacremoses.MosesTokenizer()
train_data_indices, train_labels = featurize(train_texts, train_labels, tokenizer, vocab)
test_data_indices, test_labels = featurize(test_texts, test_labels, tokenizer, vocab)

100%|██████████| 3750/3750 [00:00<00:00, 5848.10it/s]


# DataLoaders

In [0]:
# Create DataLoaders
class HyperpartisanDataset(Dataset):
    """
    Class that represents a train/validation/test dataset that's readable for PyTorch
    Note that this class inherits torch.utils.data.Dataset
    """

    def __init__(self, data_list, target_list, sequence_length):
        """
        @param data_list: list of data tokens 
        @param target_list: list of data targets 

        """
        self.data_list = data_list
        self.target_list = target_list
        self.sequence_length = sequence_length
        assert (len(self.data_list) == len(self.target_list))

    def __len__(self):
        return len(self.data_list)
        
    def __getitem__(self, key, sequence_length=None):
        """
        Triggered when you call dataset[i]
        """
        if sequence_length is None:
            sequence_length = self.sequence_length
        token_idx = self.data_list[key][:sequence_length]
        label = self.target_list[key]
        return [token_idx, label]

    def collate_func(self,batch):
        """
        Customized function for DataLoader that dynamically pads the batch so that all 
        data have the same length
        """ 
        data_list = [] # store padded sequences
        label_list = []

        if len(max(self.data_list,key=len)) < self.sequence_length:
          max_batch_seq_len = len(max(self.data_list,key=len))
        else:
           max_batch_seq_len = self.sequence_length # the length of longest sequence in batch
                                                      # if it is less than self.sequence_length.
                                                      # else max_batch_seq_len = self.sequence_length 
        for row in batch:
          if len(row[0]) < max_batch_seq_len:
            row[0] += [0] * (max_batch_seq_len - len(row[0])) # pad the sequence with 0 if it's shorter than max_batch_seq_len
          row[0] = row[0][:self.sequence_length] # only trim the sequence longer than self.sequence_length
          data_list.append(row[0])
          label_list.append(row[1]) 
        data_list = torch.LongTensor(data_list)
        label_list = torch.LongTensor(label_list)

        return [data_list, label_list]

In [0]:
train_dataset = HyperpartisanDataset(train_data_indices, train_labels, sequence_length)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size,
                                           collate_fn=train_dataset.collate_func,
                                           shuffle=False)

test_dataset = HyperpartisanDataset(test_data_indices, test_labels, sequence_length)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                           batch_size=batch_size,
                                           collate_fn=test_dataset.collate_func,
                                           shuffle=False)

In [0]:
data_batch, labels = next(iter(train_loader))
print("data batch dimension: ", data_batch.size())
print("data_batch: ", data_batch)
print("labels: ", labels)

data batch dimension:  torch.Size([50, 61])
data_batch:  tensor([[3755,   47,   16,  ...,    0,    0,    0],
        [9124, 3073, 1536,  ...,    0,    0,    0],
        [ 523,  975, 1122,  ...,    0,    0,    0],
        ...,
        [2925,   47, 8707,  ...,    0,    0,    0],
        [ 142, 4390,  467,  ...,    0,    0,    0],
        [8322,    3,  733,  ...,    0,    0,    0]])
labels:  tensor([0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0,
        1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1,
        1, 1])


In [0]:
# Initialize values
for i, item in enumerate(op_parameters):
    exec("%s = %f" % (item['var_name'],item['var_ini']))

# LSTMClassifier

In [0]:
# Define BiLSTM Model
class LSTMClassifier(nn.Module):
    """
    LSTMClassifier classification model
    """
    def __init__(self, embeddings, hidden_dim, num_layers, num_classes, bidirectional, dropout_prob):
        super().__init__()
        self.hidden_dim = hidden_dim
        # self.embedding_layer = self.load_pretrained_embeddings(embeddings)
        # self.lstm = nn.LSTM( # LSTM layer
        #                     self.embedding_layer.embedding_dim,\
        #                     hidden_size = hidden_dim, \
        #                     num_layers = num_layers, \
        #                     bidirectional = bidirectional, \
        #                     batch_first = True,\
        #                     dropout=dropout_prob)
        # self.softmax = nn.LogSoftmax() 
        # self.dropout_layer = nn.Dropout(p=dropout_prob) # dropout some units of the hidden layer
        # self.hidden2out = nn.Linear(hidden_dim*(int(bidirectional)+1), num_classes) # classifier layer
        
        self.embedding_layer = self.load_pretrained_embeddings(embeddings)
        
        self.dropout = dropout_prob
        self.lstm = nn.LSTM(self.embedding_layer.embedding_dim, hidden_size= self.hidden_dim, \
                            num_layers = num_layers, bidirectional = bidirectional, \
                            batch_first = True, dropout = self.dropout)
        self.non_linearity = nn.ReLU() # For example, ReLU
        self.clf = nn.Linear(self.hidden_dim*(int(bidirectional)+1), num_classes) # classifier layer

    def load_pretrained_embeddings(self, embeddings):
        """
           The code for loading embeddings from Lab 2
           Unlike lab, we are not setting `embedding_layer.weight.requires_grad = False`
           because we want to finetune the embeddings on our data
        """
        embedding_layer = nn.Embedding(embeddings.shape[0], embeddings.shape[1], padding_idx=0)
        embedding_layer.weight.data = torch.Tensor(embeddings).float()
        return embedding_layer
    
    def forward(self, inputs):

        # Embedding 
        # Output dim: [batch_size, seq_length, embedding_dim]
        embedded = self.embedding_layer(inputs)

        # BiLSTM
        # Output dim: [batch_size, seq_length, hidden_dim * num_directions]
        h_0 = torch.zeros(num_layers*(int(bidirectional)+1), inputs.size(0), int(self.hidden_dim))
        c_0 = torch.zeros(num_layers*(int(bidirectional)+1), inputs.size(0), int(self.hidden_dim))
        bilstm_out, _ = self.lstm(embedded, (h_0, c_0))
        
        # # Dropout layer
        # # Output dim: [batch_size, hidden_dim * num_directions]
        # dropout_out = self.dropout_layer(bilstm_out[:, -1, :])

        # # Get output from hidden states
        # # Output dim: [batch_size, num_classes]
        # linear_out = self.hidden2out(dropout_out)   
       
        # # Outputting using softmax layer
        # # Output dim: [batch_size, num_classes]
        # prob = self.softmax(linear_out)

        # return prob
               
        # Average pooling
        # Output dim: [batch_size, num_directions * hidden_size]
        num_valid = (inputs == (0 or 1)) \
            .float().sum(dim=1).clamp(1) # non-padding, non-unknown
        averaged_out = bilstm_out.sum(dim=1) / num_valid.view(-1, 1)
        
        # Non-linearity (ReLU)
        # Output dim: [num_directions * hidden_size , num_classes]
        non_linearity_out = self.non_linearity(averaged_out)

        # Linear, returns logits 
        # Output dim: [batch_size, num_classes]
        logits = self.clf(non_linearity_out)

        return logits


# Training 

In [0]:
# Training
from sklearn.metrics import accuracy_score

def evaluate(model, dataloader, device):
    accuracy = None
    model.eval()
    with torch.no_grad():
      all_preds = []
      all_labels = []
      for batch_text, batch_labels in dataloader:
          preds = model(batch_text.to(device))
          all_preds.append(preds.detach().cpu().numpy())
          all_labels.append(batch_labels.detach().cpu().numpy())
      all_preds = np.concatenate(all_preds, axis=0)
      pred_labels = all_preds.argmax(-1)
      all_labels = np.concatenate(all_labels, axis=0)
      accuracy = accuracy_score(all_labels,pred_labels)

    return accuracy 

def train(num_epoch,\
          optimizer,\
          train_loader,\
          test_loader,
          model,\
          criterion,\
          device,\
          outpath):
   
    acc = 0
    train_loss = []
    train_acc = []
    test_acc = []
    print("training")
    for epoch in tqdm(range(num_epoch)):
        outfile = open(outpath+"/bilstm_output_!.txt",'a')

        # train the model and back-propagate by batch process
        for i, (data_batch, batch_labels) in enumerate(train_loader):
            preds = model(data_batch.to(device))
            loss = criterion(preds, batch_labels.to(device))
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            train_loss.append(loss.item())

        # get train accuracy for each epoch
        acc = evaluate(model, train_loader, device)
        print('Training Accuracy of the network on epoch %i: %f %%' % (epoch, acc*100))
        train_acc.append(acc)
        training_acc = acc

        # get test accuracy for each epoch
        acc = evaluate(model, test_loader, device)
        print('Testing Accuracy of the network on epoch %i: %f %%' % (epoch, acc*100))
        test_acc.append(acc)
        testing_acc = acc
        
        # # save model afer each epoch
        # if torch.cuda.device_count() <= 1:
        #     state_dict_model = model.state_dict()
        # else:
        #     state_dict_model = model.module.state_dict()

        # state = {
        # 'epoch': epoch,
        # 'state_dict': state_dict_model,
        # 'optimizer': optimizer.state_dict(),
        # }
        
        # torch.save(state, outpath+"/"+str(epoch)+".pt")

        outfile.write("Epoch: {}, Training Accuracy: {:.4f}, Validation Accuracy: {:.4f}\n".format(epoch,training_acc,testing_acc))
        outfile.close()
    return train_loss, train_acc, test_acc

In [0]:
results = {}

In [0]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
outpath = '/content/drive/My Drive/dataset/hyperpartisan_scorer/results'
criterion = nn.CrossEntropyLoss()
torch.manual_seed(1234)

In [0]:
torch.cuda.is_available()

False

In [20]:
for item in op_parameters:
    best_acc = 0.0
    best_value = 0.0
    process = {}
    for value in item['var_range']:
        exec("%s = %f" % (item['var_name'],value))
        print("Testing %f of %s" % (value, item['var_name']))
        if torch.cuda.is_available():
            bilstm = LSTMClassifier(embeddings,hidden_dim = int(hidden_dim),num_layers = num_layers,num_classes = num_classes,dropout_prob=dropout,bidirectional = bidirectional).cuda()
        else:
            bilstm = LSTMClassifier(embeddings,hidden_dim = int(hidden_dim),num_layers = num_layers,num_classes = num_classes,dropout_prob=dropout, bidirectional = bidirectional)
        bilstm.train()
        optimizer = torch.optim.Adam(bilstm.parameters(), lr=lr)   # define a optimizer for backpropagation

        train_loss, train_acc, test_acc = train(num_epoch,optimizer,train_loader,test_loader,bilstm,criterion,device,outpath)

        process[value] = [sum(test_acc)/len(test_acc),sum(train_acc)/len(train_acc)]
        if best_acc < sum(test_acc)/len(test_acc):
            best_acc = sum(test_acc)/len(test_acc)
            best_value = value
            print("New best: ",best_acc)

    print("The best %s dimension is %f. Accuracy is %f" % (item['var_name'],best_value,best_acc))
    
    results[item['var_name']+'_best'] = best_value
    results[item['var_name']+'_process'] = process
    exec("%s = %f" % (item['var_name'],best_value))


Testing 0.100000 of lr


/usr/local/lib/python3.6/dist-packages/torch/nn/modules/rnn.py:50: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  "num_layers={}".format(dropout, num_layers))
  0%|          | 0/5 [00:00<?, ?it/s]

training
Training Accuracy of the network on epoch 0: 51.466667 %


 20%|██        | 1/5 [03:36<14:24, 216.02s/it]

Testing Accuracy of the network on epoch 0: 50.800000 %
Training Accuracy of the network on epoch 1: 74.126667 %


...

Testing Accuracy of the network on epoch 2: 70.586667 %
Training Accuracy of the network on epoch 3: 93.313333 %


 80%|████████  | 4/5 [10:56<02:44, 164.38s/it]

Testing Accuracy of the network on epoch 3: 68.240000 %
Training Accuracy of the network on epoch 4: 97.000000 %


100%|██████████| 5/5 [13:40<00:00, 164.11s/it]

Testing Accuracy of the network on epoch 4: 68.986667 %
The best dropout dimension is 0.000000. Accuracy is 0.705600


In [0]:
import json
with open(outpath+'/optimize_bilstm.json', 'w') as outfile2:
    json.dump(results, outfile2)